### Model for optimizing player transfers
### Newcastle United 2022

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, PULP_CBC_CMD, makeDict, lpSum
pd.set_option('display.float_format', '{:.1f}'.format)

In [2]:
# set up parameters
transfer_budget_dict = {'Manchester United':30e6, 'Leeds United':10e6, 'Newcastle United':200e6}
current_year = 2022
current_team = 'Newcastle United' 
team_country = 'England'
max_team_size = 25
max_foreign_size = 17
max_in = max_team_size // 4
max_transfer_budget = transfer_budget_dict[current_team]
max_old = 1

In [3]:
# prepare player data
df = pd.read_csv('player_stats_10000.csv', index_col=0)
# add index to name since player names are not unique
df['name'] = df['name'] + ' (' + (np.arange(df.shape[0])).astype(str) + ')' 
# consolidate/simplify positions
df['position'].replace('RW', 'RM', inplace=True)
df['position'].replace('ST', 'CF', inplace=True)
df['position'].replace('LW', 'LM', inplace=True)
df['position'].replace('LWB', 'LB', inplace=True)
df['position'].replace('RWB', 'RB', inplace=True)

# ability function
df['ability'] = np.where(df['age'] < 25,
                         (df['potential'] + df['overall'])/2, 
                         df['overall'])

# flag older players
df['old'] = (df['age'] >= 32).astype(int)

# flag players that have a low probability of transferring
df['low_proba'] = 0
# condition 1: just signed new contract
df.loc[(df['start'] == current_year), 'low_proba'] = 1
# condition 2: on long term contract of 4 or more years 
df.loc[(df['expiration'] - current_year >= 4), 'low_proba'] = 1
# condition 3: "marquee" outfield player
marquee_age_outfield = (df['age'] >= 22) & (df['age'] <= 32) 
marquee_contract = (df['expiration'] - current_year >= 3) 
marquee_ability = (df['ability'] >= np.percentile(df['ability'], 97)) 
df.loc[marquee_age_outfield & marquee_contract & marquee_ability & (df['position'] != 'GK'), 'low_proba'] = 1
# condition 4: "marquee" goalkeeper
marquee_age_gk = (df['age'] >= 25) & (df['age'] <= 35) 
marquee_contract = (df['expiration'] - current_year >= 3) 
marquee_ability = (df['ability'] >= np.percentile(df['ability'], 97)) 
df.loc[marquee_age_gk & marquee_contract & marquee_ability & (df['position'] == 'GK'), 'low_proba'] = 1

# estimate wage for free agents = mean wage for ability
df_signed = df[~df['team'].isna()].copy()
mean_wage_dict = df_signed.groupby(['ability'])['wage'].mean().round(-3).to_dict()
df.loc[df['team'].isna(), 'wage'] = df.loc[df['team'].isna(), 'ability'].map(mean_wage_dict)

# separate out current team and transfer pool and get more team stats
df_team = df[df['team'] == current_team].copy()
current_team_size = len(df_team)
df_pool = df[df['team'] != current_team].copy()

# current team info
df_team

,name,age,country,position,overall,potential,team,start,expiration,value,wage,ability,old,low_proba
125,K. Trippier (125),30,England,RB,84,84,Newcastle United,2022,2024,36500000.0,63000.0,84.0,0,1
314,A. Saint-Maximin (314),24,France,LM,81,86,Newcastle United,2019,2026,39000000.0,46000.0,83.5,0,1
374,Bruno Guimarães (374),23,Brazil,CDM,80,85,Newcastle United,2022,2026,29000000.0,39000.0,82.5,0,1
424,M. Dúbravka (424),32,Slovakia,GK,80,80,Newcastle United,2018,2025,10500000.0,37000.0,80.0,1,0
626,C. Wilson (626),29,England,CF,79,79,Newcastle United,2020,2024,17500000.0,53000.0,79.0,0,0
1011,M. Targett (1011),25,England,LB,77,80,Newcastle United,2022,2022,13000000.0,52000.0,77.0,0,1
1093,C. Wood (1093),29,New Zealand,CF,77,77,Newcastle United,2022,2024,10500000.0,45000.0,77.0,0,1
1206,M. Almirón (1206),27,Paraguay,CAM,76,76,Newcastle United,2019,2024,8000000.0,37000.0,76.0,0,0
1342,R. Fraser (1342),27,Scotland,RM,76,76,Newcastle United,2020,2025,8000000.0,37000.0,76.0,0,0
1352,I. Hayden (1352),26,England,CDM,76,79,Newcastle United,2016,2026,9000000.0,34000.0,76.0,0,1


In [4]:
# set up parameters, continued
max_wage_budget = df_team['wage'].sum() * 1.05
max_mean_team_age = df_team['age'].mean()

In [5]:
# print summary of parameters
print('Summary of parameters:')
print(f'Team = {current_team} ({team_country})')
print(f'Current year = {current_year}')
print(f'Max team size = {max_team_size}')
print(f'Max number of foreign players = {max_foreign_size}')
print(f'Max transfer in = {max_in}')
print(f'Transfer budget = {max_transfer_budget}')
print(f'Wage budget = {max_wage_budget}')
print('Max mean age of team = {:.2f}'.format(max_mean_team_age))
print(f'Max transfer in, age >= 32 = {max_old}')
print('Current team: {} players, sum utility = {:.2f}, mean utility = {:.2f}'
      .format(len(df_team), df_team['ability'].sum(), df_team['ability'].mean()))
print('Current team: {} players have a low probability of transfer'.format(df_team['low_proba'].sum()))

print('\n# players in transfer pool (excluding current team) = {}'.format(len(df_pool)))

Summary of parameters:
Team = Newcastle United (England)
Current year = 2022
Max team size = 25
Max number of foreign players = 17
Max transfer in = 6
Transfer budget = 200000000.0
Wage budget = 977550.0
Max mean age of team = 27.00
Max transfer in, age >= 32 = 1
Current team: 28 players, sum utility = 2126.50, mean utility = 75.95
Current team: 10 players have a low probability of transfer

# players in transfer pool (excluding current team) = 10007


In [6]:
# data prep for model
# transfer value, position, ability, wage 
dict_value_pool = dict(zip(df_pool['name'], df_pool['value']))
dict_position_pool = dict(zip(df_pool['name'], df_pool['position']))
dict_ability_pool = dict(zip(df_pool['name'], df_pool['ability']))
dict_wage_pool = dict(zip(df_pool['name'], df_pool['wage']))

dict_value_team = dict(zip(df_team['name'], df_team['value']))
dict_position_team = dict(zip(df_team['name'], df_team['position']))
dict_ability_team = dict(zip(df_team['name'], df_team['ability']))
dict_wage_team = dict(zip(df_team['name'], df_team['wage']))

# nationality
df_team['country'] = np.where(df_team['country'] == team_country, 0, 1)
df_pool['country'] = np.where(df_pool['country'] == team_country, 0, 1)
dict_country_pool = dict(zip(df_pool['name'], df_pool['country']))
dict_country_team = dict(zip(df_team['name'], df_team['country']))

# age
dict_age_pool = dict(zip(df_pool['name'], df_pool['age']))
dict_age_team = dict(zip(df_team['name'], df_team['age']))

# low_proba 
dict_lowproba_pool = dict(zip(df_pool['name'], df_pool['low_proba']))
dict_lowproba_team = dict(zip(df_team['name'], df_team['low_proba']))

# old players
dict_old_pool = dict(zip(df_pool['name'], df_pool['old']))

In [7]:
# define variables
x = LpVariable.dicts(name='b', indices=df_pool['name'],
                     lowBound=0, upBound=1, cat='Integer')
y = LpVariable.dicts(name='s', indices=df_team['name'],
                     lowBound=0, upBound=1, cat='Integer')

# define the problem
prob = LpProblem('Problem', LpMaximize)

# precompute linear combinations
# transfer value, ability, wage 
sum_value_pool = lpSum([dict_value_pool[kk] * x[kk] for kk in dict_value_pool])
sum_wage_pool = lpSum([dict_wage_pool[kk] * x[kk] for kk in dict_wage_pool])
sum_ability_pool = lpSum([dict_ability_pool[kk] * x[kk] for kk in dict_ability_pool])

sum_value_team = lpSum([dict_value_team[kk] * y[kk] for kk in dict_value_team])
sum_wage_team = lpSum([dict_wage_team[kk] * (1-y[kk]) for kk in dict_wage_team])
sum_ability_team = lpSum([dict_ability_team[kk] * (1-y[kk]) for kk in dict_ability_team])

# position
df_pos_pool = pd.get_dummies(df_pool['position'])
position_names_pool = df_pos_pool.columns
position_vector_pool = df_pos_pool.apply(lambda x: x.to_list(), axis=1).to_list()
dict_position_pool = makeDict([position_names_pool, df_pool['name']], np.array(position_vector_pool).T)

df_pos_team = pd.get_dummies(df_team['position'])
df_pos_team = df_pos_team.T.reindex(position_names_pool).T.fillna(0)
position_names_team = df_pos_team.columns
position_vector_team = df_pos_team.apply(lambda x: x.to_list(), axis=1).to_list()
dict_position_team = makeDict([position_names_team, df_team['name']], np.array(position_vector_team).T)

dict_position_need = {kk:0 for kk in position_names_pool}
dict_position_need['CF'] = 2
dict_position_need['LM'] = 1
dict_position_need['CM'] = 1
dict_position_need['CAM'] = 1
dict_position_need['CDM'] = 1
dict_position_need['RM'] = 1
dict_position_need['LB'] = 1
dict_position_need['RB'] = 1
dict_position_need['CB'] = 2
dict_position_need['GK'] = 2
assert sum(list(dict_position_need.values())) <= max_team_size, 'position needs infeasible!'

dict_position_max = {kk:0 for kk in position_names_pool}
dict_position_max['CF'] = 5
dict_position_max['LM'] = 5
dict_position_max['CM'] = 5
dict_position_max['CAM'] = 5
dict_position_max['CDM'] = 5
dict_position_max['RM'] = 5
dict_position_max['LB'] = 5
dict_position_max['RB'] = 5
dict_position_max['CB'] = 5

# nationality
sum_country_team = lpSum([dict_country_team[kk] * (1-y[kk]) for kk in dict_country_team])
sum_country_pool = lpSum([dict_country_pool[kk] * x[kk] for kk in dict_country_pool])

# age
sum_age_pool = lpSum([dict_age_pool[kk] * x[kk] for kk in dict_age_pool])
sum_age_team = lpSum([dict_age_team[kk] * (1-y[kk]) for kk in dict_age_team])

# low proba
sum_lowproba_pool = lpSum([dict_lowproba_pool[kk] * x[kk] for kk in dict_lowproba_pool])
sum_lowproba_team = lpSum([dict_lowproba_team[kk] * y[kk] for kk in dict_lowproba_team])

# old players
sum_old_pool = lpSum([dict_old_pool[kk] * x[kk] for kk in dict_old_pool])

# define constraints
# C1: maximum number of players in the team
prob += lpSum(x) - lpSum(y) <= max_team_size - current_team_size

# C2: maximum number of foreign players in the team
prob += sum_country_pool + sum_country_team <= max_foreign_size 

# C3: center forward, 2 <= n <= 5
# C4: left midfielder, 1 <= n <= 5
# C5: center midfielder, 1 <= n <= 5
# C6: center attacking midfielder, 1 <= n <= 5
# C7: center defensive midfielder, 1 <= n <= 5
# C8: right midfielder, 1 <= n <= 5
# C9: left back, 1 <= n <= 5
# C10: right back, 1 <= n <= 5
# C11: center back, 2 <= n <= 5
# C12: goalkeeper: n == 2

# minimum number of players in each position (all positions except goalkeeper)
for i in [xx for xx in dict_position_pool if xx != 'GK']: # all positions but GK
    prob += (
        lpSum(dict_position_pool[i][p] * x[p] for p in dict_position_pool[i]) + 
        lpSum(dict_position_team[i][p] * (1-y[p]) for p in dict_position_team[i])
        >= dict_position_need[i]
    )

# number of players at goalkeeper
for i in ['GK']: # restriction on GK
    prob += (
        lpSum(dict_position_pool[i][p] * x[p] for p in dict_position_pool[i]) + 
        lpSum(dict_position_team[i][p] * (1-y[p]) for p in dict_position_team[i])
        == dict_position_need[i]
    )

# maximum number of players in each position (all positions except goalkeeper)
for i in [xx for xx in dict_position_pool if xx != 'GK']: # all positions but GK
    prob += (
        lpSum(dict_position_pool[i][p] * x[p] for p in dict_position_pool[i]) + 
        lpSum(dict_position_team[i][p] * (1-y[p]) for p in dict_position_team[i])
        <= dict_position_max[i]
    )

# C13: do not consider low probability transfers (explained above)
prob += sum_lowproba_pool + sum_lowproba_team == 0    
    
# C14: maximum number of "old" players
prob += sum_old_pool <= max_old

# C15: mean age of the team should not increase after transfers
prob += (sum_age_pool + sum_age_team) <= max_mean_team_age * (current_team_size + lpSum(x) - lpSum(y))

# C16: transfer fee budget constraint
prob += sum_value_pool - sum_value_team <= max_transfer_budget 

# C17: wage constraint
prob += sum_wage_pool + sum_wage_team <= max_wage_budget

# C18: maximum number of transfers in   
prob += lpSum(x) <= max_in

# define objective function
prob += sum_ability_pool + sum_ability_team 

# solve the problem
prob.solve(PULP_CBC_CMD(msg=0))
print(f'status = {LpStatus[prob.status]}')

# print the results
for variable in prob.variables():
    if variable.varValue != 0:
        print(f'{variable.name} = {variable.varValue}')
    
print(f'Objective = {value(prob.objective)}')

status = Optimal
b_A._Davies_(100) = 1.0
b_Gavi_(1917) = 1.0
b_J._Koundé_(138) = 1.0
b_M._Neuer_(10) = 1.0
b_R._Gravenberch_(673) = 1.0
b_T._Hernández_(101) = 1.0
s_C._Clark_(2579) = 1.0
s_D._Gayle_(3267) = 1.0
s_E._Krafth_(2416) = 1.0
s_F._Fernández_(1831) = 1.0
s_J._Lascelles_(1786) = 1.0
s_M._Gillespie_(5918) = 1.0
s_M._Ritchie_(1871) = 1.0
s_Manquillo_(1872) = 1.0
s_P._Dummett_(2509) = 1.0
Objective = 1983.5


In [8]:
# final team
final_team = []
for variable in prob.variables():
    if (variable.name.startswith('s')) & (variable.varValue == 0):
        final_team.append(variable.name)
    if (variable.name.startswith('b')) & (variable.varValue == 1):
        final_team.append(variable.name)
final_team = [xx[2:].replace('_', ' ') for xx in final_team]

name_map = dict(zip(df.loc[df['name'].str.contains('-'), 'name'].str.replace('-', ' ').tolist(),
                    df.loc[df['name'].str.contains('-'), 'name'].tolist()))
final_team = [xx if xx not in name_map else name_map[xx] for xx in final_team]

df_final = df[df['name'].isin(final_team)]
print(df_final.shape)
df_final

(25, 14)


,name,age,country,position,overall,potential,team,start,expiration,value,wage,ability,old,low_proba
10,M. Neuer (10),35,Germany,GK,90,90,FC Bayern München,2011,2023,13500000.0,86000.0,90.0,1,0
100,A. Davies (100),20,Canada,LB,84,89,FC Bayern München,2019,2025,58500000.0,70000.0,86.5,0,0
101,T. Hernández (101),23,France,LB,84,89,AC Milan,2019,2024,59500000.0,51000.0,86.5,0,0
125,K. Trippier (125),30,England,RB,84,84,Newcastle United,2022,2024,36500000.0,63000.0,84.0,0,1
138,J. Koundé (138),22,France,CB,83,89,Sevilla FC,2019,2024,53000000.0,33000.0,86.0,0,0
314,A. Saint-Maximin (314),24,France,LM,81,86,Newcastle United,2019,2026,39000000.0,46000.0,83.5,0,1
374,Bruno Guimarães (374),23,Brazil,CDM,80,85,Newcastle United,2022,2026,29000000.0,39000.0,82.5,0,1
424,M. Dúbravka (424),32,Slovakia,GK,80,80,Newcastle United,2018,2025,10500000.0,37000.0,80.0,1,0
626,C. Wilson (626),29,England,CF,79,79,Newcastle United,2020,2024,17500000.0,53000.0,79.0,0,0
673,R. Gravenberch (673),19,Netherlands,CM,78,90,Ajax,2018,2023,33000000.0,11000.0,84.0,0,0


In [9]:
# player summary
set1 = set(df_team['name'])
set2 = set(df_final['name'])
df_player_summary = pd.concat([pd.DataFrame({'out':sorted(set1-set2)}),
                               pd.DataFrame({'in':sorted(set2-set1)}),
                              ], axis=1) 
df_player_summary

,out,in
0,C. Clark (2579),A. Davies (100)
1,D. Gayle (3267),Gavi (1917)
2,E. Krafth (2416),J. Koundé (138)
3,F. Fernández (1831),M. Neuer (10)
4,J. Lascelles (1786),R. Gravenberch (673)
5,M. Gillespie (5918),T. Hernández (101)
6,M. Ritchie (1871),NaN
7,Manquillo (1872),NaN
8,P. Dummett (2509),NaN


In [10]:
# team comparison
tmp = []
for xx in [df_team, df_final]:
    if xx['country'].dtype=='int':
        dict1 = {'team_size':len(xx), 'foreign':np.sum(xx['country'] == 1), 'age':xx['age'].mean(),
            'ability_sum':xx['ability'].sum(), 'ability_mean':xx['ability'].mean()}
    else:
        dict1 = {'team_size':len(xx), 'foreign':np.sum(xx['country'] != team_country), 'age':xx['age'].mean(),
                 'ability_sum':xx['ability'].sum(), 'ability_mean':xx['ability'].mean()}
    dict2 = xx['position'].value_counts().to_dict()
    tmp.append({**dict1, **dict2})
df_team_summary = pd.DataFrame(tmp)
df_team_summary['team'] = ['before_transfer', 'after_transfer']
df_team_summary[list(dict_position_need.keys())] = df_team_summary[dict_position_need.keys()].fillna(0).astype(int)
df_team_summary.insert(0, 'team', df_team_summary.pop('team'))
df_team_summary

,team,team_size,foreign,age,ability_sum,ability_mean,CB,CF,RB,LB,CDM,GK,CAM,CM,LM,RM
0,before_transfer,28,16,27.0,2126.5,75.9,6,5,4,3,2,2,2,2,1,1
1,after_transfer,25,16,25.2,1983.5,79.3,3,4,2,4,2,2,2,4,1,1


In [11]:
# ability, fee, wage breakdown
df_transfer_summary = pd.DataFrame({'ability_add':sum_ability_pool.value(),
                                    'ability_keep':sum_ability_team.value(),
                                    'ability_total':sum_ability_pool.value() + sum_ability_team.value(),
                                    'transfer_expenditure':sum_value_pool.value(),
                                    'transfer_income':sum_value_team.value(),
                                    'transfer_net':sum_value_pool.value() - sum_value_team.value(),
                                    'wage_add':sum_wage_pool.value(),
                                    'wage_keep':sum_wage_team.value(),
                                    'wage_total':sum_wage_pool.value() + sum_wage_team.value()},
                                  index=[0])
df_transfer_summary

,ability_add,ability_keep,ability_total,transfer_expenditure,transfer_income,transfer_net,wage_add,wage_keep,wage_total
0,514.0,1469.5,1983.5,227000000.0,27100000.0,199900000.0,263000.0,680000.0,943000.0
